# Sur les avantages familiaux

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from openfisca_matplotlib import graphs

In [2]:
from datetime import date  # module nécessaire pour la définition des dates, dont notamment les dates de naissances
import openfisca_france    # module décrivant le système socio-fiscal français

TaxBenefitSystem = openfisca_france.init_country()  # Initialisation de la classe décrivant le système socio-fiscal français
tax_benefit_system = TaxBenefitSystem()  # Création d'une instance du système socio-fiscal français 

In [3]:
salaire_imposable_maximal = 80000

In [4]:
def couple_avec_enfants(nombre_enfants = 0, year = 2014):
    enfant = [dict(
        birth = date(2005, 1, 1),
        )]
    enfants = enfant * nombre_enfants
    simulation = tax_benefit_system.new_scenario().init_single_entity(
        axes = [
            dict(
                count = 100,
                min = 0,
                max = salaire_imposable_maximal * 3,
                name = 'sal',
                ),
            ],
        period = "{}:3".format(year-2),
        parent1 = dict(
            birth = date(1980, 1, 1),
            ),
        parent2 = dict(
            birth = date(1980, 1, 1)
            ),
        enfants = enfants,
        menage = dict(
            loyer = 1000 * 3,
            so = 4,
            ),
        ).new_simulation(debug = True)
    return simulation

## Gain lié à l'enfant marginal

In [5]:
def gain_enfant_marginal(rang_enfant, year = 2014):
    assert rang_enfant >= 1
    situation_initiale = couple_avec_enfants(rang_enfant-1, year)
    situation_finale = couple_avec_enfants(rang_enfant, year)
    revdisp_initial = situation_initiale.calculate("revdisp", period = year)
    revdisp_final = situation_finale.calculate("revdisp", period = year)
    return revdisp_final - revdisp_initial

Je reprends le travail de Brian qui a effectué le travail pas à pas (pédestrement comme dirait Mahdi).

Il crée un premier dictionnaire vide. Puis il associe dans le dictionnaire à chaque clé 'year' le gain marginal d'un enfant en utilisant la fonction gain_enfant_marginal (qui pour rappel nous renvoie un array numpy du gain pour les ménages allant de 0 à 80 000 euros).

In [6]:
gain_by_year_margin_0_1 = dict()

In [7]:
for year in range(2010, 2015):
    gain_by_year_margin_0_1[year] = gain_enfant_marginal(1, year = year)
#print gain_by_year_margin_0_1

Il fait parreil pour le gain d'un a 2 enfants.

In [8]:
gain_by_year_margin_0_2 = dict()

In [9]:
for year in range(2010, 2015):
    gain_by_year_margin_0_2[year] = gain_enfant_marginal(2, year = year)
#print gain_by_year_margin_0_2

Pour 3.

In [10]:
gain_by_year_margin_0_3 = dict()

In [11]:
for year in range(2010, 2015):
    gain_by_year_margin_0_3[year] = gain_enfant_marginal(3, year = year)
#print gain_by_year_margin_0_3

Il a créé trois dictionnaire associant à chaque clé année un array numpy.

### Liste

Ça ne mange pas de pain, mais on peut mettre le trois dictionnaires obtenus au sein d'une liste. Comme ça on n'a plus qu'un seul objet contenant tous nos gains marginaux à 1, 2 et 3 enfants.

In [12]:
liste_des_gains_marginaux = [gain_by_year_margin_0_1, gain_by_year_margin_0_2, gain_by_year_margin_0_3]
#liste_des_gains_marginaux

### Dictionnaire de dictionnaires :

Notre liste ne nous indique pas quel gains marginal est dedans, si on oublie dans quel ordre on a mis les gains marginaux on est marron.

Mais il suffit de créer un dictionnaire qui aura pour clé le gain marginal concidéré !

In [13]:
dict_gains_marginaux = {'1 enfant' : gain_by_year_margin_0_1, '2 enfants' : gain_by_year_margin_0_2, '3 enfants' : gain_by_year_margin_0_3}
#dict_gains_marginaux

## Fini le pas à pas, maintenant on peut voler !

Maintenant qu'on a trouvé la meilleur forme pour stocker nos gains marginaux ( si on a bien suvit un dictionnaire avec des clés pour le nombre d'enfants associés a des dictionnaires dont les clés sont les années qui sont associés a des array numpy contenant le gain marginal pour des ménages allant de 0 à 80 000 euros, ouf ! ). On peut bien réduire notre code en faisant deux boucles for imbriquées ! 

In [16]:
gains_marginaux = dict() #je défini un dictionnaire qui contiendra l'ensemble des mes gains marginaux
for nombre_enfant in range(1,6):  #je fais varier mon nombre d'enfants de 1 à 5
    temporary_dict = dict()    # je crée un dictionnaire temporaire vide
    for year in range(2010, 2015):
        temporary_dict[year] = gain_enfant_marginal(nombre_enfant, year = year) #je rempli ce dict vide avec mes années
    gains_marginaux['{} enfants'.format(nombre_enfant)] = temporary_dict #je met mon dict temporaire correspondant a un gain marginal pour un nombre d'enfant précis dans un grand dico contenant tout mes gains marginaux. 

In [17]:
print gains_marginaux

{'1 enfants': {2010: array([ 1805.30761719,  1805.30761719,  1805.30859375,  1805.30664062,
        1805.30664062,  1805.30859375,  1805.30761719,  1805.30664062,
        1805.30761719,  1995.24316406,  2249.171875  ,  2286.78125   ,
        2329.765625  ,   968.59863281,   742.93261719,  1982.81933594,
        1982.81835938,  2131.43945312,  2284.12695312,  2437.0234375 ,
        2589.7109375 ,  2659.3203125 ,  2740.59960938,  2782.40625   ,
        2824.2734375 ,  2822.98046875,  2772.0859375 ,  2940.578125  ,
        2582.61132812,  2257.625     ,  1950.5546875 ,  1643.484375  ,
        1055.65429688,   669.5625    ,   669.5625    ,   669.5625    ,
         669.5625    ,   747.80273438,   864.1328125 ,   980.46484375,
        1096.9609375 ,  1213.29101562,  1329.625     ,  1445.95703125,
        1562.29101562,  1678.78515625,  1795.1171875 ,  1911.44921875,
        2027.78515625,  2144.11328125,  2260.609375  ,  2336.        ,
        2336.        ,  2336.        ,  2336.        ,  

Et voilà on a notre dictionnaire de dictionnaires contenant toute l'information que l'on souhaire avoir ! :)

Je peux aussi faire un dictionnaire un peu différent qui me donnerait en clé les années associé a des dictionnaires contenant le numéro de l'enfant marginal en clé associé au vecteur numpy. TODO ?!